![MIoT_GDPI](img/MIOT_GDPI_header.png)

# Unidad 02 - Operaciones comunes en Stream Learning

El objetivo principal de esta práctica es que os familiaricéis con las operaciones básicas para trabajar en *online learning* y como integrarlas en un *pipeline* de [River](https://riverml.xyz/). Podréis comprobar que su uso es similar al de `scikit-learn` que ya conocéis.

La mayor parte del contenido de este Notebook se dedica a explicar el uso de los comandos y de su integración en un *pipeline* apoyándose en ejemplos concretos que ilustran su aplicación a un problema real. Es crucial que dediquéis tiempo a leer y comprender el material, en lugar de simplemente ejecutar el código. Os invitamos a experimentar modificando y variando el código proporcionado para que podáis explorar las distintas opciones y profundizar en su funcionamiento.



**importante**: El Notebook contiene varios ejercicios sencillos. Debéis desarrollarlos durante la clase y enviarlos por el aula virtual del curso, en la tarea correspondiente.



## Referencias útiles para la práctica
1. API Pandas: [https://pandas.pydata.org/docs/reference/index.html](https://pandas.pydata.org/docs/reference/index.html)
2. Api River: [https://riverml.xyz](https://riverml.xyz)
3. Api Scikit-Learn: [https://scikit-learn.org/stable/api/index.html](https://scikit-learn.org/stable/api/index.html)
4. Bahri, M., Bifet, A., Gama, J., Gomes, H. M., & Maniu, S. (2021). [Data stream analysis: Foundations, major tasks and tools](https://doi.org/10.1002/widm.1405). Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery, 11(3), e1405.
5. Gomes, H. M., Read, J., Bifet, A., Barddal, J. P., & Gama, J. (2019). [Machine learning for streaming data: state of the art, challenges, and opportunities](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://kdd.org/exploration_files/3._CR_7._Machine_learning_for_streaming_data_state_of_the_art-Final.pdf). ACM SIGKDD Explorations Newsletter, 21(2), 6-22.


## 1. Avanzando de *batch ML* tradicional al *online ML*


Este tutorial está inspirado en los ejemplos proporcionados por [River](https://riverml.xyz/)en su página [web](https://riverml.xyz/latest/recipes/reading-data/)  Por lo tanto, os animamos  a consultar la documentación original para una exploración más profunda de cada uno de los temas particulares que se muestren a continuación.

En general, casi cualquier enfoque asociado al aprendizaje automático contiene los siguientes pasos:
1. Entender y definir el problema y su contexto.
2. Obtener los datos.
3. Explorar, analizar y entender los datos.
4. Preprocesar los datos
5. Seleccionar, optimizar y entrenar los modelos ML.
6. Evaluar y el modelo seleccionado.
7. Desplegar, monitorizar y mantener la solución.

Antes de comenzar, vamos a examinar un ejemplo clásico que utiliza todos estos pasos utilizando la librería  `scikit-learn`:

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from rich import print


# Carga de datos
dataset = load_breast_cancer()
X, y = dataset.data, dataset.target

# Preparación del pipeline con los pasos de preprocesado y el modelo seleccionado
pipe = Pipeline([
    ('scale', StandardScaler()),                       # Estandarización de los datos avg 0 y std 1
    ('extractor', PCA(0.95)),                          # Reducción de la dimensionalidad con PCA (0.95 of the varianza)
    ('classifier', LogisticRegression(solver='lbfgs')) # Selección y configuración del modelo
])

# Definición de una validación cruzada estableciendo una semilla
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluación del modelo para cada fold de la validación cruzada
scorer = make_scorer(roc_auc_score)
scores = cross_val_score(pipe, X, y, scoring=scorer, cv=cv)

# Imprime la media de la métrica y su desciación estándard
print(f'ROC AUC: {scores.mean():.4f} (± {scores.std():.4f})')

¿Cuál es el problema con esta aproximación? En realidad, no tiene ningún problema, pero existen algunas desventajas potenciales, particularmente con respecto al tamaño del conjunto de los datos, la aparición de datos nuevos y la heterogeneidad de las características a lo largo del tiempo.

* **Tamaño del conjunto de datos**: imagina que los datos para el entrenamiento fueran demasiado grandes para caber en la memoria de tu ordenador. En ese caso, el programa, probablemente,  se habría bloqueado. Aunque se pueden aplicar diferentes técnicas para minimizar este problema,  existe un límite para estas optimizaciones. De hecho, si el dataset constara de millones de observaciones (ej.  cientos de gigabytes), podría haberse requerido hardware especial solo para cargar el dataset.

* **Datos nuevos**: Otro problema potencial es la incorporación de  datos  nuevos al modelo. Los enfoques tradicionales requieren comenzar a entrenar desde cero con un nuevo conjunto de datos resultante de la combinación de los datos antiguos con las nuevas observaciones disponibles. Esto es particularmente problemático en aplicaciones en tiempo real donde tienes  datos nuevos disponibles cada poco. En muchas aplicaciones reales, la solución es desarrollar un pipeline de integración continua que pueda generar y desplegar un nuevo modelo cada poco. es importante destacar que por datos nuevos nos referimos a observaciones que son radicalmente diferentes de las empleadas para entrenar, dónde las relaciones entre las entradas (*inputs*) y salidas (*outputs*) ha cambiado de forma que el modelo no puede gestionarlas.

* **Heterogeneidad de características**: Finalmente, otra desventaja con el enfoque tradicional es que las características con las que se ha entrenado deberán estar disponibles siempre durante el tiempo de producción. Además, no pueden incorporarse nuevas características a lo largo del tiempo. En escenarios de plantas industriales es habitual que determinadas características no estén disponibles en un momento determinado (sensores en estropeados o con mal funcionamiento) para luego estar nuevamente disponibles. En estos mismos escenarios, nuevos sensores pueden desplegarse y empezar a medir y generar nuevas características. Desde un punto de vista tradicional, la única forma de gestionar un número cambiante de características es reentrenando desde cero.


## 2. *Incremental Learning*

Como se mencionó en la unidad anterior, el aprendizaje incremental (*incremental learning*) también se conoce como aprendizaje en línea (*online learning*) o aprendizaje de flujo (*stream learning*). Es habitual que se sustituya el término *online* por incremental o *stream* debido a lo confuso del concepto de "aprendizaje en línea", que, generalmente,  se refiere a una opción educativa (esto es bastante obvio si buscas en Google "aprendizaje en línea"). 

El concepto de *incremental learning* se refiere a ajustar un modelo ML a partir de las observaciones obtenidas a través de un flujo (*stream*) de datos en tiempo real. Expresado de otra manera, los datos no están disponibles en su totalidad como en el ML tradicional (dataset de entrenamiento), sino que las observaciones se proporcionan una por una a medida que se leen de un flujo de datos. Por ejemplo, imagina el caso anterior que, en lugar de disponer del conjunto de datos completo para entrenar, tenemos un punto de referencia temporal que proporciona una observación a la vez. Esto se puede simular con un simple bucle: 


In [ ]:
for xi, yi in zip(X, y):#simula obtener una observación por cada vuelta del bucle
    pass

print(xi)#Imprimimos la última observación para ver su formato
print(yi)#Imprimimos la última etiqueta para ver su formato

Dado que los datos ya están en memoria, este no es el escenario ideal pero, sin embargo, tened en cuenta que en este caso particular solo tenemos acceso a una sola muestra cada vez (dentro del bucle y representada por los valores xi, yi). Este bucle simula el flujo de datos (*data stream*), donde la función zip devuelve un iterador de Python. Esto no es demasiado diferente de la forma en que podríamos iterar a través de un archivo CSV, recibir un stream de datos de Kafka u obtener los resultados de una consulta SQL, fila por fila.

Una cosa a tener en cuenta es el hecho de que en este ejemplo `xi` es una instancia de `numpy.array` (los datos se cargaron utilizando la librería de scikit-learn) pero, por su diseño, la librería  `River`, utiliza la clase `dict` como base para su trabajo trabajo. Cada observación viene representada por un diccionario y se asume que cada una de ellas equivale a una muestra del *stream* de datos.

Hay algunas consideraciones que vale la pena recordar y tener en cuenta:

1. El  uso de `numpy` es interesante en entornos de computación de alto rendimiento. Recuerda que `dict` está implementado en Python, mientras que `numpy.array` está implementado a bajo nivel en C y Fortran. Una de las ventajas de usar `dict` es su facilidad de uso y que facilita la legibilidad del programa.

2. El procesamiento en línea es diferente al procesamiento por lotes (*batch learning*), ya que **la vectorización no aporta ninguna aceleración en el proceso** (recordad que solo estamos procesando una observación de cada vez). Las librerías de procesamiento numérico esán optimizadas para operaciones vectorizadas e introducen una sobrecarga considerable si solo se procesa una única muestra. En este tipo de escenarios, el uso de  `dict` no penaliza y puede simplificar el desarrollo.

In [ ]:
print(f"In numpy.array format:{xi}\n")
print(f"In dict format: {dict(zip(dataset.feature_names, xi))}")

Para facilitar la portabilidad de algoritmos y *pipelines* trabajo de ML preexistentes, `River` proporciona una función *wrapper* que transforma los datasets de `scikit-learn` al formato requerido. 

In [ ]:
from river import stream
for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer()):
    pass

print(xi)#Imprimimos la última observación para ver su formato
print(yi)#Imprimimos la última etiqueta para ver su formato

Si bien la mayoría de las operaciones se traducen con bastante facilidad entre el ML tradicional y el aprendizaje incremental, no todas se pueden hacer de forma directa. Vamos a ejemplificar esto con el problema del escalado de los datos, en concreto con la estandarización (escalar los datos para que tengan media 0 y varianza 1). Esta es una operación trivial sobre lotes de datos pero que en aprendizaje incremental requerirá calcular estadísticas dinámicamente. 

 se debe tener cuidado al convertir entre los bien conocidos batches (lotes) y streams (flujos). Ilustraremos esto considerando el problema del escalado de datos. Recordemos que la estandarización de los datos, es decir, escalar los datos para que tengan media 0 y varianza 1, es una operación trivial sobre lotes, mientras que para los flujos de datos se requerirán estadísticas en ejecución (running) o móviles (moving). Esta sencilla operación se vuelve un poco más complicada en el aprendizaje incremental porque no conocemos los valores de la media y la desviación estándar antes de procesar todos los datos.

Una primera aproximación podría ser realizar una primera pasada sobre todos los datos para calcular los valores necesarios y luego escalar los valores durante una segunda pasada. Sin embargo, esta solución es inconsistente con nuestro objetivo: procesar los datos una sola vez.

La solución  es usar estadísticas dinámicas donde no se usen  la media y la desviación estándar exactas, sino una estimación que se actualiza con cada nuevo valor. Más formalmente, dado la media $\mu_t$, y el conteo de observacionbes  $n_t$, ambos en el instante temporal $t$, la media móvil puede ser facilmente actualizada para cada muestra aplicando la siguiente función: 


$$\large
  n_{t+1} = n_t +1 \\   $$
$$  
  \large
  \mu_{t+1} = \mu_t +\frac{x - \mu_t}{n_{t+1}}    
$$

De la misma manera, el cálculo de la varianza ($\sigma_t$) en el instante $t$ podría actualizarse empleando:
$$
\large
  s_{t+1} = s_t + (x-\mu_t)\times(x-\mu_{t+1})\\
  \large
  \sigma_{t+1} = \frac{s_{t+1}}{n_{t+1}}
$$

 Estas fórmulas pueden ser fácilmente implementadas en Python. Veamos un ejemplo:

In [ ]:

n, mean, s, variance = 0, 0, 0, 0

for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer()):
    n += 1
    mean_t = mean
    mean += (xi['mean radius'] - mean_t) / n
    s += (xi['mean radius'] - mean_t) * (xi['mean radius'] - mean)
    variance = s / n

    print(f'Media dinámica: {mean:.3f} - Varianza dinámica: {variance:.3f}')
    
print(f'Media final: {mean:.3f} - Varianza final: {variance:.3f}')

Comparemos ahora el resultado con la implementación nativa de `numpy`

In [ ]:
import numpy as np
i = list(dataset.feature_names).index('mean radius')
print(f'True mean: {np.mean(X[:, i]):.3f}')
print(f'True variance: {np.var(X[:, i]):.3f}')

Como era de esperar, los resultados finales  son idénticos, con una diferencia clave: la implementación de `numpy` requiere que todos los datos estén disponibles para el cálculo, mientras que en el caso *online*, se calculan progresivamente. Por lo tanto, debemos ser conscientes de estos comportamientos y darnos cuenta de que los resultados con solo unas pocas observaciones no son precisos.

Aunque la mayoría de las medidas estadísticas  se podrían desarrollar fácilmente en Python de forma dinámica, la realidad es que no es necesario, ya que River proporciona la mayoría de ellas en el módulo `stats`. Por ejemplo, para crear tanto la media móvil como la varianza móvil emplearíamos:

In [ ]:
from river import stats

r_mean=stats.Mean()
r_variance=stats.Var()

for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer()): 
    r_mean.update(xi['mean radius'])
    r_variance.update(xi['mean radius'])
    print(f'Running mean: {r_mean.get():.3f} - Running variance: {r_variance.get():.3f}')

Una vez que sabemos cómo calcular las diferentes estadísticas móviles sobre los datos, el siguiente paso es usarlas para estandarizar las observaciones de manera similar al enfoque tradicional. En `River`, se, dentro del módulo `preprocessin`están disponibles varias funciones  para aplicar este tipo de operaciones. Por ejemplo, para estandarizar todas las características del ejemplo anterior, podríamos realizar:



In [ ]:
from river.preprocessing import StandardScaler

scaler = StandardScaler()

for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer()):
    scaler.learn_one(xi)
    xi_scaled=scaler.transform_one(xi)
    print(f"xi:{xi}")
    print(f"scaled xi {xi_scaled}")